## SCD Tipo 2 aplicado a la tabla `usuarios`

En este notebook exploramos cómo aplicar el concepto de Slowly Changing Dimensions (SCD) Tipo 2 a la tabla `usuarios`, con el objetivo de preservar el historial de cambios en los datos.

### Eleccion de SCD: Tipo 2

Con el modelo de datos proveído no hay columnas que operen con Slowly Changing Dimensions, pero de implementarse un buen punto de partida seria Usuarios, y SCD tipo 2. De esta forma podriamos **preservar el historial** de los cambios, analizar evolucion de los datos y mantener trazabilidad histórica. La tabla Usuarios es un buen punto de partida ya que no debería tener cambios muy recurrentes que incrementen demasiado el tamaño de nuestra tabla pero si los suficientes para probar la implementacion.

In [ ]:
# Simulación de la tabla original de usuarios
import pandas as pd
from datetime import datetime

usuarios = pd.DataFrame([
    {'usuarioid': 1, 'nombre': 'Julieta', 'apellido': 'Gómez', 'dni': '12345678', 'email': 'julieta@correo.com', 'contraseña': 'C213'},
    {'usuarioid': 2, 'nombre': 'Carlos', 'apellido': 'Pérez', 'dni': '87654321', 'email': 'carlos@correo.com', 'contraseña': 'C123'}
])

usuarios

In [ ]:
# Agregamos columnas para SCD Tipo 2
usuarios_dim = usuarios.copy()
usuarios_dim['fecha_inicio'] = datetime(2023, 1, 1)
usuarios_dim['fecha_fin'] = pd.NaT
usuarios_dim['is_current'] = True

usuarios_dim

### Simulación de un cambio


In [ ]:
# Cerramos la versión actual y agregamos una nueva fila con el nuevo email
nueva_version = {
    'usuarioid': 1,
    'nombre': 'Julieta',
    'apellido': 'Gómez',
    'dni': '12345678',
    'email': 'julieta.nuevo@example.com',
    'contraseña': 'hash1',
    'fecha_inicio': datetime(2024, 3, 1),
    'fecha_fin': pd.NaT,
    'es_actual': True
}

# Actualizamos la versión anterior
usuarios_dim.loc[(usuarios_dim['usuarioid'] == 1) & (usuarios_dim['es_actual']), 'fecha_fin'] = nueva_version['fecha_inicio']
usuarios_dim.loc[(usuarios_dim['usuarioid'] == 1) & (usuarios_dim['es_actual']), 'es_actual'] = False

# Agregamos la nueva versión
usuarios_dim = pd.concat([usuarios_dim, pd.DataFrame([nueva_version])], ignore_index=True)

usuarios_dim